In [ ]:
from qcodes.instrument_drivers.stanford_research.SR830 import SR830

In [ ]:
### Adjust autoranging
def autorange_sr830s(sr830s):
    '''
    Takes a list of sr830s to autorange
    Autoranging is performed based on the phase on the first provided lockin
    TODO: set sensitivity bounds
    '''
    if len(sr830s) == 0: return
    p = sr830s[0].P() # phase of current measurement
    if np.abs(p)<160: return
    for s in sr830s[1:]:
        def autorange_once(sr830):
            r = sr830.R()
            sens = sr830.sensitivity()
            if r > 0.9 * sens:
                return sr830.increment_sensitivity()
            elif r < 0.1 * sens:
                return sr830.decrement_sensitivity()
            else: return False
        sets = 0
        while autorange_once(s) and sets < 3:
            sets += 1
            # Sleep 10 times the time constant between range sets.
            # The manual has more detailed guidelines for how long to wait.
            time.sleep(10 * s.time_constant())